Models were automatically reconstructed from NCBI RefSeq accession codes with carveme.

For NJ4:

```
carve --refseq GCF_003014985.1 -o GEMs/NJ4.xml
```


In [1]:
from cobra.io import read_sbml_model

nj4 = read_sbml_model('GEMs/NJ4.xml')

In [10]:
# useful tool for seeing the intercondections between metabolites!

from utils import model_validation as modval

metabolites = ["xylu__D_c", "xu5p__D_c"]
G = modval.create_graph(nj4, metabolites)
modval.visualise_graph(G, 'nj4_xylose')

nj4_xylose.html


In [11]:
nj4.reactions.TKT1

Reaction identifier,TKT1
Name,Transketolase
Memory address,0x1238d8490
Stoichiometry,r5p_c + xu5p__D_c <=> g3p_c + s7p_c Alpha-D-Ribose 5-phosphate + D-Xylulose 5-phosphate <=> Glyceraldehyde 3-phosphate + Sedoheptulose 7-phosphate
GPR,WP_010964657_1
Lower bound,-1000.0
Upper bound,1000.0


In [4]:
# checking if production of a metabolite is possible at SS

from utils import model_validation as modval

modval.production_possible(nj4, "EX_etoh_e")

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


True